<a href="https://colab.research.google.com/github/chunribu/manim4stats/blob/main/2022/05/variance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 样本方差为什么要除以n-1

**观看视频**

[![YouTube](https://icon-icons.com/downloadimage.php?id=168737&root=2699/PNG/32/&file=youtube_logo_icon_168737.png)](https://youtu.be/tSBk9sJ25JQ) 
[![BiliBili](https://icon-icons.com/downloadimage.php?id=206792&root=3261/PNG/32/&file=bilibili_logo_icon_206792.png)](https://www.bilibili.com/video/BV1kU4y1m7uV/) 
[![iXigua](https://icon-icons.com/downloadimage.php?id=154582&root=2596/PNG/32/&file=xigua_icon_154582.png)](https://www.ixigua.com/7099248726815965732)


In [ ]:
# 安装Manim
!sudo apt install libcairo2-dev ffmpeg \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython --upgrade

In [ ]:
from manim import *

Manim Community v0.15.2

In [ ]:
colors = [BLUE, TEAL,  GREEN, YELLOW, GOLD, RED, MAROON]

def get_dots():
    dots = [Dot(color=i) for i in colors]
    labels = [SingleStringMathTex(f'x_{i+1}', color=colors[i], font_size=20) for i in range(7)]
    dots_g = VGroup(*dots, *labels).arrange_in_grid(rows=2)
    box = SurroundingRectangle(dots_g, color=WHITE, buff=0.3)
    dots_g_b = VGroup(box, dots_g)
    return dots, dots_g_b

def get_nl():
    return NumberLine(
        x_range=[1,7,1],
        include_numbers=True
    )

In [ ]:
%%manim -ql -v WARNING View0

class View0(Scene):
    def construct(self):
        dots, dots_g_b = get_dots()
        self.play(Create(dots_g_b), run_time=2)
        self.wait(2)
        self.play(dots_g_b.animate.to_edge(UL, buff=MED_LARGE_BUFF))

        nl = get_nl()
        self.play(Write(nl))

        _dots = [d.copy() for d in dots]
        self.play(*[d.animate.move_to(nl.n2p(i+1)) for i,d in enumerate(_dots)])
        self.play(*[Indicate(d,color=WHITE) for d in _dots])

        nl_d = VGroup(nl, *_dots)
        self.play(nl_d.animate.shift(DOWN*2))

        labels = [nl.get_number_mobject(i) for i in range(1,8)]
        for i,l in enumerate(labels):
            self.play(Indicate(l, color=colors[i], scale_factor=1.5), run_time=0.4)

        t_n_pre = MathTex('N=')
        t_n = labels[-1].copy().scale(1.5)
        t_n_g = VGroup(t_n, t_n_pre)
        self.play(Write(t_n_pre))
        self.wait()
        self.play(
            t_n.animate.next_to(t_n_pre, RIGHT), 
            Circumscribe(t_n, shape=Circle, buff=0.4, run_time=0.3)
        )
        self.play(t_n_g.animate.next_to(dots_g_b, DOWN, buff=SMALL_BUFF))

        t_mu_pre = MathTex(r'\mu=')
        self.play(Write(t_mu_pre))
        self.play(t_mu_pre.animate.shift(LEFT*4))

        t_mu_list = [t_mu_pre]
        for i, l in enumerate(labels):
            t_mu_list.append(l.copy().scale(1.5))
            self.play(
                t_mu_list[-1].animate.next_to(t_mu_list[-2]),
                Circumscribe(_dots[i], shape=Circle, buff=0.4, run_time=0.3),
                run_time=0.5
            )
            if i < 6:
                plus = MathTex('+').next_to(t_mu_list[-1])
                t_mu_list.append(plus)
                self.play(Create(t_mu_list[-1]), run_time=0.5)

        t_mu_x_g = VGroup(*t_mu_list[1:])
        ul = Underline(t_mu_x_g).shift(UP*0.4)
        self.play(t_mu_x_g.animate.shift(UP*0.5), Create(ul))
        t_n_ = t_n.copy()
        self.play(t_n_.animate.next_to(ul, DOWN))
        t_mu_su = MathTex('=4').next_to(ul, RIGHT)
        self.play(Write(t_mu_su))

        t_mu_list.extend([ul, t_n_, t_mu_su])
        t_mu_g = VGroup(*t_mu_list)
        t_mean = MathTex(r'\mu=\frac{1}{N} \sum_{i=1}^N x_{i}=4').shift(LEFT*2)
        self.play(ReplacementTransform(t_mu_g, t_mean))
        self.wait()
        self.play(t_mean.animate.next_to(t_n_g, DOWN))

        mean_coord = nl.n2p(4)
        dl_mu = DashedLine(UP*2, DOWN).move_to(mean_coord+UP*0.6)
        self.play(Create(dl_mu))
        dl_mu_t = MathTex(r'\mu').next_to(dl_mu, DOWN)
        self.play(Write(dl_mu_t))
        
        d_list = []
        for i in range(7):
            _hight = (4 - abs(3-i)) * 0.5
            p1 = nl.n2p(i+1) + UP*_hight
            p2 = mean_coord + UP*_hight
            if i==3:
                d_list.append(Square(side_length=0.2, color=colors[i]).move_to(p1).rotate(PI/4))
            else:
                d_list.append(
                    DoubleArrow(p1, p2, buff=0, tip_length=0.2, color=colors[i])
                )
            self.play(Write(d_list[-1]))

        t_sigma = MathTex('(','3','^2+','2','^2+','1','^2+','0','^2+','1','^2+','2','^2+','3','^2',')',r'\times\frac{1}{7}','=4')\
                  .shift(UP*2+RIGHT*2)
        for i in range(7):
            d_sym = d_list[i].copy()
            d_t = t_sigma.submobjects[i*2+1]
            self.play(d_sym.animate.move_to(d_t.get_center()), run_time=0.6)
            self.play(FadeOut(d_sym, run_time=0.5), Write(d_t))
        self.play(*[Write(t_sigma.submobjects[s-1]) for s in [3,5,7,9,11,13,15]])
        self.play(*[Write(t_sigma.submobjects[s]) for s in [0,15]])
        self.play(*[Write(t_sigma.submobjects[s]) for s in [-2]])
        self.play(*[Write(t_sigma.submobjects[s]) for s in [-1]])

        t_variance = MathTex(r'\sigma^2=\frac{1}{N}\sum_{i=1}^N (x_{i}-\mu)^2=4').shift(UP*2)
        self.play(ReplacementTransform(t_sigma, t_variance))
        self.play(t_variance.animate.next_to(t_mean.copy().shift(RIGHT*0.7), DOWN))

        nl_g = VGroup(*([nl_d, dl_mu, dl_mu_t] + d_list + labels))
        self.play(nl_g.animate.scale(0.5)\
                              .to_edge(UR, buff=MED_LARGE_BUFF))

        self.play(t_mean.animate.move_to(ORIGIN+UP))
        self.play(t_variance.animate.move_to(ORIGIN+DOWN))

        self.wait(2)

In [ ]:
%%manim -ql -v WARNING View1

class View1(Scene):
    def pos_xn(self, m):
        left = m.get_left()
        x2 = left + UP*0.28 + RIGHT*1.6
        x3 = left + UP*0.28 + RIGHT*4.2
        x4 = left + UP*0.28 + RIGHT*6.9
        n = left + DOWN*0.35 + RIGHT*4.9
        return x2, x3, x4, n
    
    def get_rectangle_corners(self, bottom_left, top_right):
        return [
            (top_right[0], top_right[1]),
            (bottom_left[0], top_right[1]),
            (bottom_left[0], bottom_left[0]),
            (top_right[0], bottom_left[0]),
        ]

    def construct(self):
        dots, dots_g_b = get_dots()
        nl = get_nl()
        self.add(dots_g_b.to_edge(UL, buff=MED_LARGE_BUFF))
        self.play(Write(nl), run_time=1.5)
        self.wait()
        t_smp = Text('抽样(Sampling)').scale(0.5).to_edge(DOWN, buff=MED_LARGE_BUFF)
        self.play(Write(t_smp))
        _dots = [dots[i].copy() for i in [1,2,3]]
        self.play(*[_dots[i].animate.move_to(nl.n2p(i+2)) for i in range(3)], run_time=1.5)
        self.play(FadeOut(t_smp))

        nl_d = VGroup(nl, *_dots)
        self.play(nl_d.animate.shift(DOWN*2))
        self.wait()

        t_n = MathTex('n=3')
        self.play(Write(t_n),run_time=1.5)
        self.play(t_n.animate.next_to(dots_g_b, DOWN))
        self.wait()

        t_ss = MathTex(r'S^2 = \frac{(x_2-\mu)^2+(x_3-\mu)^2+(x_4-\mu)^2}{n}')
        t_ss_ = MathTex(r'=\frac{(2-\mu)^2+(3-\mu)^2+(4-\mu)^2}{3}').shift(UP*1.8+RIGHT*2)
        self.play(Write(t_ss), run_time=2)
        self.play(t_ss.animate.shift(UP*3+RIGHT*2))

        p_x2, p_x3, p_x4, p_n = self.pos_xn(t_ss)
        p_targets = [p_x2, p_x3, p_x4, p_n]
        p_params = [d.copy() for d in _dots]+[t_n.copy()]
        for i, p in enumerate(p_params):
            self.play(p.animate.move_to(p_targets[i]), run_time=0.6)
            self.play(Flash(p), FadeOut(p))
        self.wait()
        self.play(FadeIn(t_ss_))
        self.wait()

        t_ss___ = MathTex(r'=\mu^2-6\mu+\frac{29}{3}').shift(UP*0.5+RIGHT*0.15)
        self.play(FadeIn(t_ss___))
        self.wait()

        t_ss__ = MathTex(r'= \frac{3\mu^2 - 2(x_2+x_3+x_4)\mu + (x_2^2+x_3^2+x_4^2)} {n}')\
                    .scale(0.86)\
                    .shift(UP*3+RIGHT*2.5)
        
        t_ss_final_r = MathTex(r'S^2= \frac{3}{n}\mu^2 - 2\frac{x_2+x_3+x_4}{n}\mu + \frac{x_2^2+x_3^2+x_4^2}{n}')\
                        .move_to(t_ss__.get_center())\
                        .scale(0.86)
        t_ss_final_l = MathTex(r'S^2=\mu^2-6\mu+\frac{29}{3}').move_to(t_ss___.get_center())
        self.add(t_ss_final_l)
        self.play(t_ss_final_l.animate.scale(0.75).next_to(t_n, DOWN))
        self.wait(2)

        self.add(t_ss_final_r)
        t_ss_g = VGroup(t_ss, t_ss_, t_ss___)
        self.play(ReplacementTransform(t_ss_g, t_ss_final_r))
        self.play(t_ss_final_r.animate.next_to(dots_g_b, RIGHT, buff=MED_LARGE_BUFF))
        self.wait()

        self.play(FadeOut(nl_d), run_time=0.5)

#
        ax = Axes(
            x_range=[0, 7],
            y_range=[0, 7],
            x_length=5,
            y_length=4.5,
            axis_config={
                "include_numbers": True,
                "include_tip": False
            },
        ).shift(RIGHT)

        graph = ax.plot(
            lambda x: x**2 - 6*x + 29/3,
            color=BLUE,
            x_range=[0.5, 5.5, 0.01],
            use_smoothing=False,
        )

        x = ValueTracker(1)

        dot = Dot()
        dot.add_updater(lambda m: m.move_to(ax.c2p(
                x.get_value(),
                x.get_value()**2 - 6*x.get_value() + 29/3
            )))
        dot.set_z_index(10)

        def get_rectangle():
            polygon = Polygon(
                *[
                    ax.c2p(*i)
                    for i in self.get_rectangle_corners(
                        (0, 0), (x.get_value(), x.get_value()**2 - 6*x.get_value() + 29/3)
                    )
                ]
            )
            polygon.stroke_width = 1
            polygon.set_stroke(GRAY_D)
            return polygon
        polygon = always_redraw(get_rectangle)

        _t_mu = MathTex(r'\mu', color=BLUE_A)
        _t_mu.add_updater(lambda m: m.move_to(ax.c2p(
                x.get_value(), 
                -1
            )))
        _t_mu.set_z_index(9)

        _t_ss = MathTex(r'S^2', color=BLUE_A)
        _t_ss.add_updater(lambda m: m.move_to(ax.c2p(
                -1, 
                x.get_value()**2 - 6*x.get_value() + 29/3
            )))
        _t_ss.set_z_index(9)

        self.wait()

        self.play(Create(ax), run_time=1.5)
        self.wait()
        self.play(Create(graph), run_time=1.5)
        self.wait()
        self.play(Create(dot), Create(polygon), Create(_t_mu), Create(_t_ss))
        self.wait()

        self.play(x.animate.set_value(5), run_time=3)
        self.play(x.animate.set_value(4), run_time=1.5)
        self.wait(2)

        dl = DashedLine(UP*4, DOWN).move_to(ax.c2p(3,3.5))
        self.play(Write(dl))
        self.wait()

        t_2ab = MathTex(r'\frac{b}{-2a}').move_to(UR*1.2)
        t_2ab_ = MathTex(r'=\frac{\;}{-2\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;}').next_to(t_2ab, RIGHT).shift(DOWN*0.2)
        t_a = MathTex(r'\frac{3}{n}', color=RED).scale(0.86).move_to(t_ss_final_r.get_center()+LEFT*2.9)
        t_b = MathTex(r'-2\frac{x_2+x_3+x_4}{n}', color=GREEN).scale(0.86).move_to(t_ss_final_r.get_center()+LEFT*0.6)
        self.play(Create(t_2ab))
        self.play(Create(t_2ab_))
        self.wait()
        
        self.add( t_a, t_b)
        self.play(t_a.animate.move_to(t_2ab_.get_center()+DOWN*0.3+RIGHT*0.3))
        self.play(t_b.animate.move_to(t_2ab_.get_center()+UP*0.75+RIGHT*0.2))
        self.wait(2)

        def get_slopes():
            return ax.get_secant_slope_group(
                x=x.get_value(),
                graph=graph,
                dx=0.01,
                secant_line_length=2,
                secant_line_color=WHITE,
            )
        slopes = always_redraw(get_slopes)
        self.play(Create(slopes))
        self.play(x.animate.set_value(3), run_time=2)

        t_2ab__ = MathTex(r'=\overline{x}').next_to(t_2ab_, RIGHT)
        self.play(FadeIn(t_2ab__))
        t_2ab_g = VGroup(t_2ab, t_2ab_, t_2ab__, t_a, t_b)
        t_xbar = MathTex(r'\overline{x}').move_to(UR*1.2+LEFT*0.2)
        self.play(ReplacementTransform(t_2ab_g, t_xbar))
        self.wait(2)

        # self.play(t_xbar.animate.set_color(BLUE))
        self.play(_t_mu.animate.set_color(GRAY_E), Flash(_t_mu, color=GRAY, flash_radius=0.2))
        self.wait(4)

        self.play(Indicate(_t_ss, color=WHITE))
        self.wait()

        t_cmp = MathTex(r'\frac{\sum (x-\overline{x})^2}{n} < \frac{\sum (x-\mu)^2}{N}').shift(RIGHT*4)
        self.play(Write(t_cmp), run_time=3)
        self.play(ApplyWave(t_cmp, direction=RIGHT))

        self.wait(4)

In [ ]:
%%manim -ql -v WARNING View2

class View2(Scene):
    def construct(self):
        t_ss = MathTex(r'S^2 = \frac{\sum_{i=1}^{n}(x_i-\overline{x})^2}{\;}')
        t_n = MathTex(r'n').move_to(t_ss.get_center()+DOWN*0.5+RIGHT*0.6)
        t_minus1 = MathTex(r'-1').scale(0.8).next_to(t_n, RIGHT, buff=0.1)
        self.play(FadeIn(t_ss), FadeIn(t_n), run_time=2)
        self.wait(2)
        
        self.play(Create(t_minus1), run_time=2)

        t_n_minus1 = VGroup(t_n, t_minus1)
        self.play(t_n_minus1.animate.shift(LEFT*0.3))

        t_ubv = Text('无偏方差(Unbiased Variance)').scale(0.5).to_edge(DOWN, buff=MED_LARGE_BUFF)
        self.play(Write(t_ubv))

        self.wait(2)